In [1]:

# import the necessary packages
from threading import Thread
import sys
import cv2
 
# import the Queue class from Python 3
if sys.version_info >= (3, 0):
    from queue import Queue
 
# otherwise, import the Queue class for Python 2.7
else:
    from Queue import Queue

In [2]:

class FileVideoStream:
    def __init__(self, path, queueSize=256):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.stream = cv2.VideoCapture(path)
        self.stopped = False

        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queueSize)
        
    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        t.start()
        return self
    
    def update(self):
        # keep looping infinitely
        while True:
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return
 
            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file
                (grabbed, frame) = self.stream.read()

                # if the `grabbed` boolean is `False`, then we have
                # reached the end of the video file
                if not grabbed:
                    self.stop()
                    return

                # add the frame to the queue
                self.Q.put(frame)


    def read(self):
        # return next frame in the queue
        return self.Q.get()

    def more(self):
        # return True if there are still frames in the queue
#         print('size', self.Q.qsize())
        return self.Q.qsize() > 0
    
    def full(self):
        return self.Q.full()

    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True
        print('done')
        
    def is_stopped(self):
        return self.stopped

    def close(self):
        self.stream.release()

In [3]:
import numpy as np
import argparse
import time
import cv2

# start the file video stream thread and allow the buffer to
# start to fill
print("[INFO] starting video file thread...")
video_file = '/media/golden/72FFC6EE48B5CF39/drone-tracking/kenya-tracking/raw-videos/raw-footage/observation088/APR08_2018_A_DJI_0005.MOV'
fvs = FileVideoStream(video_file).start()
time.sleep(1.0)
 

[INFO] starting video file thread...


In [4]:

# loop over frames from the video file stream
frame_num = 0
start = time.time()
while fvs.more():
    
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale (while still retaining 3
    # channels)
    if fvs.full() or fvs.is_stopped():
#     if True:
        if frame_num % 500 == 0:
            print(frame_num)
        frame = fvs.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)	

        # show the frame and update the FPS counter
    #     cv2.imshow("Frame", frame)

#         time.sleep(.5)
        frame_num += 1
    
print('fps', frame_num / (time.time() - start))
fvs.close()
cv2.destroyAllWindows()
fvs.stop()
    


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
done
19500
fps 38.28501565513844
done


In [3]:
import cv2
cv2.getBuildInformation()

'\nGeneral configuration for OpenCV 3.4.3 =====================================\n  Version control:               3.4.3\n\n  Extra modules:\n    Location (extra):            /io/opencv_contrib/modules\n    Version control (extra):     3.4.3\n\n  Platform:\n    Timestamp:                   2018-09-09T13:49:56Z\n    Host:                        Linux 4.4.0-101-generic x86_64\n    CMake:                       3.9.0\n    CMake generator:             Unix Makefiles\n    CMake build tool:            /usr/bin/gmake\n    Configuration:               Release\n\n  CPU/HW features:\n    Baseline:                    SSE SSE2 SSE3\n      requested:                 SSE3\n    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2\n      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX\n      SSE4_1 (3 files):          + SSSE3 SSE4_1\n      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2\n      FP16 (1 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX\n      AVX (5 fi